In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
#import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from dateutil.relativedelta import relativedelta

import sys
sys.path.append('../')
import utils


import cf_xarray as cfxr

from sklearn.linear_model import LinearRegression
import scipy.signal as signal
from scipy import stats
from datetime import timedelta

import seaborn as sns
import matplotlib as mpl
import cmocean
import cmocean.cm as cmo
from matplotlib.gridspec import GridSpec

from matplotlib.lines import Line2D
import matplotlib.patches as mpatches

import string

In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': True})

## Import G

In [3]:
G_ds_path = 'Outputs/G_pulse_ds.nc4'
G_cdr_ds_path = 'Outputs/G_cdr_ds.nc4'

G_ds = utils.import_polyfit_G(G_ds_path, G_cdr_ds_path)

In [4]:
model_run_pulse_dict = utils.model_run_pulse_dict
model_run_cdr_pulse_dict = utils.model_run_cdr_pulse_dict
model_run_control_dict = utils.model_run_picontrol_dict


model_run_4x_dict = utils.model_run_4x_dict


In [5]:
model_color = utils.model_color
type_color = utils.type_color

In [6]:
A = utils.A
ds_out = utils.ds_out

In [9]:
tas_co2_pictrl = {}
tas_4x = {}

for m in model_run_4x_dict.keys():
    tas_4x[m] = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_4x_dict[m]}',  use_cftime=True) #kg/m2/s
    lat_corners = cfxr.bounds_to_vertices(tas_4x[m].isel(time = 0)['lat_bnds'], "bnds", order=None)
    lon_corners = cfxr.bounds_to_vertices(tas_4x[m].isel(time = 0)['lon_bnds'], "bnds", order=None)
    tas_4x[m] = tas_4x[m].assign(lon_b=lon_corners, lat_b=lat_corners)
    tas_4x[m] = utils._regrid_ds(tas_4x[m], ds_out)


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter 

In [10]:
for m in model_run_control_dict.keys():
    tas_co2_pictrl[m] = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_control_dict[m]}',  use_cftime=True) #kg/m2/s
    lat_corners = cfxr.bounds_to_vertices(tas_co2_pictrl[m].isel(time = 0)['lat_bnds'], "bnds", order=None)
    lon_corners = cfxr.bounds_to_vertices(tas_co2_pictrl[m].isel(time = 0)['lon_bnds'], "bnds", order=None)
    tas_co2_pictrl[m] = tas_co2_pictrl[m].assign(lon_b=lon_corners, lat_b=lat_corners)
    tas_co2_pictrl[m] = utils._regrid_ds(tas_co2_pictrl[m], ds_out)


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter 

In [13]:
## fix the times so that they line up according to the notes above
m = 'NORESM2'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl[m]['time'] -timedelta(365*1599)


m = 'UKESM1_r1'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl['UKESM1_r1']['time'] - timedelta(360*110)

m = 'CANESM5_r1p2'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl['CANESM5_r1p2']['time']- timedelta(365*3700)

m = 'CANESM5_r1p1'
tas_co2_pictrl[m]['time'] = tas_co2_pictrl['CANESM5_r1p1']['time']- timedelta(365*3351)


In [16]:
dif_4x = {}
for m1 in model_run_4x_dict.keys():
    print(m1)
    if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
        m2 = 'UKESM1_r1'
    elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2' or m1 == 'CANESM5_r4p2' or m1 == 'CANESM5_r5p2':
         m2 = 'CANESM5_r1p2'
    else:
        m2 = m1
    print(m1, m2)
    
    dif_4x[m1] = tas_4x[m1] - tas_co2_pictrl[m2]#.isel(time = slice(yrs[m1]*12,-1))

UKESM1_r1
UKESM1_r1 UKESM1_r1
MIROC
MIROC MIROC
NORESM2
NORESM2 NORESM2
ACCESS
ACCESS ACCESS
CANESM5_r1p2
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r1p1
CANESM5_r1p1 CANESM5_r1p1


In [17]:
dif_4x = {}
for m1 in model_run_4x_dict.keys():
    print(m1)
    if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
        m2 = 'UKESM1_r1'
    elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2' or m1 == 'CANESM5_r4p2' or m1 == 'CANESM5_r5p2':
         m2 = 'CANESM5_r1p2'
    else:
        m2 = m1
    print(m1, m2)
    
    dif_4x[m1] = tas_4x[m1] - tas_co2_pictrl[m2]
    
    if len(dif_4x[m1]['time']) > 3000:  #hack to get the time stamping to work, should find better fix
        periods = 3000
    else:
        periods = len(dif_4x[m1]['time'])
        
    times = pd.date_range('2000', periods= periods, freq='MS')
    weights = times.shift(1, 'MS') - times
    weights = xr.DataArray(weights, [('time', dif_4x[m1]['time'][:periods].values)]).astype('float')
    dif_4x[m1] =  (dif_4x[m1] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')

    dif_4x[m1]['year'] = range(len(dif_4x[m1]['year']))
    

UKESM1_r1
UKESM1_r1 UKESM1_r1
MIROC
MIROC MIROC
NORESM2
NORESM2 NORESM2
ACCESS
ACCESS ACCESS
CANESM5_r1p2
CANESM5_r1p2 CANESM5_r1p2
CANESM5_r1p1
CANESM5_r1p1 CANESM5_r1p1


In [18]:
for m in dif_4x.keys():
    dif_4x[m] = dif_4x[m].drop('height')

In [19]:
#get rid of height and limit the time to the length of the GF
for m1 in ['UKESM1_r1', 'UKESM1_r2', 'UKESM1_r3', 'UKESM1_r4', 'NORESM2',
       'GFDL', 'MIROC', 'CANESM5_r1p2', 'CANESM5_r2p2', 'ACCESS', 'CANESM5_r3p2']:
    
    for t in ['pulse','cdr']:
        if m1 == 'UKESM1_r1' or m1 == 'UKESM1_r2' or m1 == 'UKESM1_r3' or m1 == 'UKESM1_r4':
            m2 = 'UKESM1_r1'
        elif m1 == 'CANESM5_r1p1' or m1 == 'CANESM5_r2p1' or m1 == 'CANESM5_r3p1':
             m2 = 'CANESM5_r1p1'
        elif m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2':
             m2 = 'CANESM5_r1p2'
        else:
            m2 = m1
    
        
        length = len(G_ds.sel(model = m2, pulse_type = t).dropna(dim = 's')['s'])
        dif_4x[m] = dif_4x[m].sel(year = slice(0,length))


In [20]:
ds_dif_4x = xr.concat([dif_4x[m] for m in dif_4x.keys()], pd.Index([m for m in dif_4x.keys()], name='model'), coords='minimal')


In [21]:
ds_dif = ds_dif_4x.rename({'year':'s'})


## Emissions profile

In [23]:
emis_profile =  xr.open_dataset(f'Outputs/4x_emis_profile_full.nc4')
emis_profile = emis_profile.rename({'__xarray_dataarray_variable__':'emis'})

## PiCtrl

In [27]:

#combine our picontrol data into one dataset, normalizing the time to year 0
pictrl = {}
for m in tas_co2_pictrl.keys():    
    times = tas_co2_pictrl[m].time.get_index('time')
    weights = times.shift(-1, 'MS') - times.shift(1, 'MS')
    weights = xr.DataArray(weights, [('time', tas_co2_pictrl[m]['time'].values)]).astype('float')
    pictrl[m] =  (tas_co2_pictrl[m] * weights).groupby('time.year').sum('time')/weights.groupby('time.year').sum('time')
    pictrl[m]['year'] = pictrl[m]['year'] - pictrl[m]['year'][0] 
    
for m in pictrl.keys():
    pictrl[m] = pictrl[m].drop('height')
ds_pictrl = xr.concat([pictrl[m] for m in pictrl.keys()], pd.Index([m for m in pictrl.keys()], name='model'), coords='minimal')


## Global Mean Analysis

In [28]:
%%time
GF = G_ds.weighted(A).mean(dim = ['lat','lon'])

conv_mean = {}
for m1 in emis_profile.model.values:
    conv_mean[m1] = {}
    for t in ['pulse','cdr']:
        m2 = 'CANESM5_r1p2'
        conv_mean[m1][t] = signal.convolve( np.array(GF.sel(model = m2, pulse_type = t).dropna(dim = 's')), np.array(emis_profile['emis'].sel(model = m1)),'full')
        conv_mean[m1][t] = utils.np_to_xr_mean(conv_mean[m1][t], GF.sel(model = m2, pulse_type = t), emis_profile.sel(model = m1))
        length = len(G_ds.weighted(A).mean(dim = ['lat','lon']).dropna(dim = 's')['s'])
        conv_mean[m1][t] = conv_mean[m1][t][:length]


CPU times: user 22.8 s, sys: 6.73 s, total: 29.5 s
Wall time: 29.6 s


In [29]:
#convert to dataset

conv_dict = {}

for m1 in conv_mean.keys():
    conv_dict[m1] = xr.concat([conv_mean[m1][t] for t in conv_mean[m1].keys()], pd.Index([t for t in conv_mean[m1].keys()], name='pulse_type'), coords='minimal')
conv_mean_ds = xr.concat([conv_dict[m1] for m1 in conv_dict.keys()], pd.Index([m1 for m1 in conv_dict.keys()], name='model'), coords='minimal')


## Spatial Analysis

In [30]:
%%time

GF = G_ds

conv = {}
for m1 in emis_profile.model.values:
    conv[m1] = {}
    for t in ['pulse','cdr']:
        m2 = 'CANESM5_r1p2'
        conv[m1][t] = signal.convolve(np.array(GF.sel(model = m2, pulse_type = t).dropna(dim = 's')), 
                                      np.array(emis_profile['emis'].sel(model = m1))[..., None, None],
                                      'full')
        conv[m1][t] = utils.np_to_xr(conv[m1][t], 
                                          GF.sel(model = m2, pulse_type = t), 
                                          emis_profile.sel(model = m1))


CPU times: user 6.87 s, sys: 3.49 s, total: 10.4 s
Wall time: 10.4 s


In [31]:
#convert to dataset

conv_dict = {}

for m1 in conv.keys():
    conv_dict[m1] = xr.concat([conv[m1][t] for t in conv[m1].keys()], pd.Index([t for t in conv[m1].keys()], name='pulse_type'), coords='minimal')
conv_ds = xr.concat([conv_dict[m1] for m1 in conv_dict.keys()], pd.Index([m1 for m1 in conv_dict.keys()], name='model'), coords='minimal')


## Model Mean Convolution

In [32]:
%%time

GF = G_ds

conv = {}
for m1 in emis_profile.model.values:
    conv[m1] = {}
    for t in ['pulse','cdr']:
        conv[m1][t] = signal.convolve(np.array(GF.sel(pulse_type = t).dropna(dim = 's').mean(dim = 'model')), 
                                      np.array(emis_profile['emis'].sel(model = m1))[..., None, None],
                                      'full')
        conv[m1][t] = utils.np_to_xr(conv[m1][t], 
                                          GF.sel(pulse_type = t).mean(dim = 'model'), 
                                          emis_profile.sel(model = m1))


CPU times: user 25.8 s, sys: 13.7 s, total: 39.5 s
Wall time: 39.6 s


In [33]:
#convert to dataset

conv_dict = {}

for m1 in conv.keys():
    conv_dict[m1] = xr.concat([conv[m1][t] for t in conv[m1].keys()], pd.Index([t for t in conv[m1].keys()], name='pulse_type'), coords='minimal')
conv_mod_mean_ds = xr.concat([conv_dict[m1] for m1 in conv_dict.keys()], pd.Index([m1 for m1 in conv_dict.keys()], name='model'), coords='minimal')


# Save out

In [34]:
conv_mean_ds.to_netcdf('Outputs/4x_conv_mean_ds.nc4')

conv_ds.to_netcdf('Outputs/4x_conv_ds.nc4')

conv_mod_mean_ds.to_netcdf('Outputs/4x_conv_ds.nc4')

emis_profile.to_netcdf('Outputs/4x_emis_profile.nc4')

ds_dif.to_netcdf('Outputs/4x_ds_dif.nc4')


In [ ]:
emis_profile